In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

In [ ]:
ROOT_DIR = '/content/drive/MyDrive/Computer_Vision/Projects'

In [ ]:
import cv2
import random
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

model_path = '/content/drive/MyDrive/Computer_Vision/Projects/runs/detect/train_output5/weights/best.pt'
model = YOLO(model_path)

video_path = '/content/drive/MyDrive/Computer_Vision/Projects/vehicles_3.mp4'

# Load the video using OpenCV
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
else:

    output_video_path = '/content/drive/MyDrive/Computer_Vision/Projects/vehicles_3_output.mp4'

    # Get the video frame rate and resolution
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or *'XVID'
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))


    confidence_threshold = 0.4  # Only show objects with confidence >= 0.4
    min_object_size = 500  # Minimum size of the object (in pixels) to keep

    # 12 random colors for different classes
    colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(12)]

    # Process  in the video
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Perform inference on the frame
        results = model(frame, conf=confidence_threshold)

        # Filter detections based on confidence and size
        boxes = results[0].boxes  # Get the boxes
        filtered_detections = []

        # Loop through the detections and filter based on confidence and size
        for box in boxes:
            if box.conf[0] >= confidence_threshold:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Get bounding box coordinates
                label = int(box.cls[0])  # Get the class label for the detected object
                confidence = box.conf[0]  # Get the confidence score for the detection
                object_area = (x2 - x1) * (y2 - y1)  # Calculate object size

                # keep detections that are large enough
                if object_area >= min_object_size:
                    filtered_detections.append((box, label, confidence, (x1, y1, x2, y2)))

        # Annotate the frame
        annotated_frame = frame.copy()
        for box, label, confidence, (x1, y1, x2, y2) in filtered_detections:
            # Select a random color for the bounding box based on the label
            color = colors[label]

            # Draw the bounding box on the frame
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 1)
            cv2.putText(annotated_frame, f'{model.names[label]} {confidence:.2f}',
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # output video
        out.write(annotated_frame)

        # Display the frame
        cv2_imshow(annotated_frame)

    # Release everything after the processing is done
    cap.release()
    out.release()
    cv2.destroyAllWindows()


Output hidden; open in https://colab.research.google.com to view.